## Melbourne Airbnb users ranking system based upon number of reviews.

# Preparing the data
To create this system we need to have a database of users urls and their number of reviews. The Melbourne users url data was retrieved from publicly available insideairbnb website http://data.insideairbnb.com/australia/vic/melbourne/2020-05-13/data/listings.csv.gz which contains all the listings in the melbourne area.

To get number of Reviews Web scrapping was needed to get latest data. Beautifulsoup web parser was used.

In [2]:
# importing packages
from bs4 import BeautifulSoup
import requests
#Pandas/numpy for data manipulation
import pandas as pd
import numpy as np
#regular expressions to find specific string from text
import re

# Extracting url data from listings.csv

In [3]:
# load the listings dataframe
# set options to see all columns
df=pd.read_csv('listings.csv',low_memory=False)    
pd.set_option('display.max_columns', 500)

In [4]:
#checking first 3 rows of dataframe
df.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,9835,https://www.airbnb.com/rooms/9835,20200513213618,2020-05-14,Beautiful Room & House,NaN,"House: Clean, New, Modern, Quite, Safe. 10Km f...","House: Clean, New, Modern, Quite, Safe. 10Km f...",none,Very safe! Family oriented. Older age group.,NaN,"YES ! The bus (305,309) is exactly two blocks ...","Kitchen, backyard, upstairs lounge. We'd like ...",As much as the guest wants.,Prefence would be given females or a couple,NaN,NaN,https://a0.muscache.com/im/pictures/44620/5a58...,NaN,33057,https://www.airbnb.com/users/show/33057,Manju,2009-08-21,AU,About us: We are an East Indian couple in our ...,NaN,NaN,0%,f,https://a0.muscache.com/defaults/user_pic-50x5...,https://a0.muscache.com/defaults/user_pic-225x...,NaN,1.0,1.0,"['email', 'phone', 'reviews']",f,f,"Bulleen, VIC, Australia",Balwyn North,Manningham,NaN,Bulleen,VIC,3105,Melbourne,"Bulleen, Australia",AU,Australia,-37.77268,145.09213,f,House,Private room,2,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,""Pets live on this property"",Do...",NaN,$60.00,NaN,NaN,NaN,NaN,1,$22.00,1,365,1,1,365,365,1.0,365.0,34 months ago,t,30,60,90,365,2020-05-14,4,0,2011-05-24,2015-09-12,90.0,9.0,10.0,10.0,10.0,9.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.04
1,10803,https://www.airbnb.com/rooms/10803,20200513213618,2020-05-15,Room in Cool Deco Apartment in Brunswick East,A large air conditioned room with queen spring...,The apartment is Deco/Edwardian in style and h...,A large air conditioned room with queen spring...,none,This hip area is a crossroads between two grea...,Due to Airbnb policies I don’t accept bookings...,Easy transport options - the tram is right out...,Wifi. Bathroom and kitchen is shared but I mos...,I'm home mostly but sometimes I may be working...,Theres a yoga studio downstairs so if you see ...,NaN,NaN,https://a0.muscache.com/im/pictures/31323790/9...,NaN,38901,https://www.airbnb.com/users/show/38901,Lindsay,2009-09-16,"Melbourne, Victoria, Australia",As an artist working in animation and video I ...,within a few hours,100%,100%,f,https://a0.muscache.com/im/users/38901/profile...,https://a0.muscache.com/im/users/38901/profile...,Brunswick,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Brunswick East, VIC, Australia",Brunswick,Moreland,NaN,B

In [5]:
#Extracting unique urls from dataset
urls=df['host_url'].unique()
urls

array(['https://www.airbnb.com/users/show/33057',
       'https://www.airbnb.com/users/show/38901',
       'https://www.airbnb.com/users/show/50121', ...,
       'https://www.airbnb.com/users/show/183028929',
       'https://www.airbnb.com/users/show/43321650',
       'https://www.airbnb.com/users/show/215618647'], dtype=object)

# Web Scrapping with BeautifulSoup

In [ ]:
# initialize an empty list of Number of Reviews for each url
Reviews=[]
#initialize for loop to iterate through each url
for i in range(0,len(urls)):
    #print after each 1000 iteration to monitor progress
    if (i%1000)==0:
        print(i)
    #initialize parser
    html = requests.get(urls[i]).text
    soup = BeautifulSoup(html, "html.parser")
    #finding all the tags containing word 'review' in the text 
    str1=soup.findAll('div', text = re.compile('review'), attrs = {'class' : "_czm8crp"})
    # if no reviews were found than store Nan as number of reviews for the url
    # otherwise extract the number found in the string and store it as a integer
    # for each url append the number of reviews in the list Reviews.
    if len(str1)==0:
        review=np.nan
    else:
        review=int(re.findall('\d+', str1[0].text)[0])
    Reviews.append(review)

In [ ]:
#check if the reviews are found for each url
len(Reviews),len(urls)

# Creating the database

In [ ]:
#create a dataset containg urls and number of reviews as columns
df_review = pd.DataFrame(list(zip(urls, Reviews)), columns =['url', 'Reviews'])
df_review

In [ ]:
#saving the dataset as csv file for future uses
df_review.to_csv('Melbourne Airbnb Reviews.csv',index=False)

In [7]:
data=pd.read_csv('Melbourne Airbnb Reviews.csv').copy()

In [8]:
#Replace nan values with zero(no reviews)
data.fillna(0,inplace=True)
#creating a rank column based upon number of reviews
#we might not need it but might be useful if we want to see specific rank of user
data['Rank']=data['Reviews'].rank(method='min',ascending=False)

In [9]:
# define a function to determine the position of user based on their reviews or rank
def position(rank):
    above=data[data['Rank']<=rank].count()[0]
    ans=(above/len(data))*100
    return ans

In [10]:
#Create a position column and apply the position function on each of element
data['Position']=data['Rank']
data['Position']=data['Position'].apply(position)

In [11]:
data

,url,Reviews,Rank,Position
0,https://www.airbnb.com/users/show/33057,4.0,10564.0,73.916506
1,https://www.airbnb.com/users/show/38901,153.0,1217.0,8.150262
2,https://www.airbnb.com/users/show/50121,1251.0,59.0,0.391584
3,https://www.airbnb.com/users/show/164193,167.0,1066.0,7.148072
4,https://www.airbnb.com/users/show/182833,175.0,1004.0,6.703392
...,...,...,...,...
15062,https://www.airbnb.com/users/show/345577446,0.0,13456.0,100.000000
15063,https://www.airbnb.com/users/show/346008039,1.0,12516.0,89.301122
15064,https://www.airbnb.com/users/show/183028929,1.0,12516.0,89.301122
15065,https://www.airbnb.com/users/show/43321650,7.0,9211.0,63.722042


In [ ]:
#Saving final dataset as csv file for easy access in future.
data.to_csv('Melbourne.csv',index=False)

# Ranking System Function

In [12]:
database=pd.read_csv('Melbourne.csv').copy()

In [13]:
def profile_rank():
    url=input('Enter the user url:')
    user_data=database[database['url']==url]
    if len(user_data)==0:
        print('User not found in the database')
    else:
        review=int(user_data['Reviews'])
        position=float(user_data['Position'])
        print('The user has {} Reviews and the user is in top {:.2f} % of Melbourne Airbnb users based on number of reviews.'.format(review,position))
        
        

In [14]:
#run this function to check for any user.
profile_rank()

Enter the user url:https://www.airbnb.com/users/show/99824610
The user has 15 Reviews and the user is in top 47.95 % of Melbourne Airbnb users based on number of reviews.
